# Leaflet web map of Last.fm artists

To see the final product live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)

Convert the geocoded CSV file of artists produced by [musicbrainz_geocoder.ipynb](musicbrainz_geocoder.ipynb) to a GeoJSON file for Leaflet web mapping.

In [1]:
import pandas as pd, json
from IPython.display import IFrame

C:\Users\Danny\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# load the geocoded data set of artists
df = pd.read_csv('data/mb_geocoded.csv', encoding='utf-8')
print '{:,} total rows'.format(len(df))
df = df[pd.notnull(df['place_latlng'])]
print '{:,} rows with lat-long'.format(len(df))
print '{:,} unique lat-longs'.format(len(df['place_latlng'].unique()))

428 total rows
298 rows with lat-long
216 unique lat-longs


In [3]:
# determine how many times each place appears in dataset, and break latlng into discrete lat and long
place_counts = df['place_full'].value_counts()
df['place_count'] = df['place_full'].map(lambda x: place_counts[x])
df['lat'] = df['place_latlng'].map(lambda x: x.split(',')[0])
df['lng'] = df['place_latlng'].map(lambda x: x.split(',')[1])
df = df[['name', 'place_full', 'place_count', 'lat', 'lng']]
df.head()

,name,place_full,place_count,lat,lng
0,Kaiser Chiefs,"Leeds, Leeds, West Yorkshire, England, United ...",1,53.8950414,-1.6405178
1,Jessie J,"London, England, United Kingdom",12,51.5073219,-0.1276473
2,Rudimental,"Hackney, London, England, United Kingdom",1,51.5432402,-0.049362
3,Thirty Seconds to Mars,"Los Angeles, Los Angeles County, California, U...",18,34.0543942,-118.2439408
6,Box Car Racer,"San Diego, San Diego County, California, Unite...",2,32.7174209,-117.1627713


In [4]:
# create html list of artists from each place
features = []
for place_full in df['place_full'].unique():
    
    # how many artists to show before saying "...and n more"
    num_to_show = 3
    line_break = u'<br />'
    artists = u''
    
    place_count = place_counts[place_full]
    names = df[df['place_full']==place_full]['name']
    
    if place_count <= num_to_show:
        for name in names:
            artists = u'{}{}{}'.format(artists, name, line_break)
            
    else:
        for name in names[0:num_to_show]:
            artists = u'{}{}{}'.format(artists, name, line_break)
        artists = u'{}...and {} more'.format(artists, place_count - num_to_show)
        
    features.append([place_full, artists])

df_leaflet = pd.DataFrame(features, columns=['place_full', 'artists'])

In [5]:
# add lat and long back to the dataframe
place_lat_lng = {}
df_unique = df[['place_full', 'lat', 'lng']].drop_duplicates(subset='place_full')
for label in df_unique.index:
    place_lat_lng[df_unique.loc[label, 'place_full']] = (df_unique.loc[label, 'lat'], df_unique.loc[label, 'lng'])
    
df_leaflet['lat'] = df_leaflet['place_full'].map(lambda x: place_lat_lng[x][0])
df_leaflet['lng'] = df_leaflet['place_full'].map(lambda x: place_lat_lng[x][1])
df_leaflet.head()

,place_full,artists,lat,lng
0,"Leeds, Leeds, West Yorkshire, England, United ...",Kaiser Chiefs<br />,53.8950414,-1.6405178
1,"London, England, United Kingdom",Jessie J<br />Florence + the Machine<br />Cold...,51.5073219,-0.1276473
2,"Hackney, London, England, United Kingdom",Rudimental<br />,51.5432402,-0.049362
3,"Los Angeles, Los Angeles County, California, U...",Thirty Seconds to Mars<br />Josh Groban<br />R...,34.0543942,-118.2439408
4,"San Diego, San Diego County, California, Unite...",Box Car Racer<br />blink‐182<br />,32.7174209,-117.1627713


## Write to GeoJSON for leaflet mapping

In [6]:
# function to write the dataframe out to geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [8]:
geojson = df_to_geojson(df_leaflet, df_leaflet.columns, lat='lat', lon='lng')

# save the geojson result to a file
output_filename = 'leaflet/lastfm-dataset.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson)))
    
# how many features did we save to the geojson file?
print '{:,} geotagged features saved to file'.format(len(geojson['features']))

217 geotagged features saved to file


In [17]:
# show the iframe of the leaflet web map here
IFrame('leaflet/lastfm-artists-map.html', width=980, height=600)

To see the final product live, visit: http://geoffboeing.com/2016/05/analyzing-lastfm-history/